In [1]:
import numpy as np
import pandas as pd
import os, sys
import pickle
import requests, bs4
from datetime import datetime, date, timedelta

In [2]:
os.chdir('/Users/ahelgeso/Documents/GitHub/bball')
os.getcwd()

'/Users/ahelgeso/Documents/GitHub/bball'

In [3]:
db = pd.read_csv('2018stats.csv')
today = datetime(2018, 10, 25)

In [4]:
# Check if dates are already datetime objects
fixdates = False
for date in db['Date']:
    if type(date) != pd._libs.tslib.Timestamp:
        fixdates = True
fixdates

True

In [5]:
# Turn dates into datetime objects
if fixdates:
    db['Date'] = db['Date'].apply(str)
    db['Date'] = pd.to_datetime(db.Date, format= "%Y%m%d")
        
# Sort db every time, i don't see why not
db = db.sort_values(by = ['Date']).reset_index(drop = True)
# Data is now ordered by date
db['Date'][-30:-1]

2430   2018-04-10
2431   2018-04-10
2432   2018-04-10
2433   2018-04-10
2434   2018-04-10
2435   2018-04-10
2436   2018-04-11
2437   2018-04-11
2438   2018-04-11
2439   2018-04-11
2440   2018-04-11
2441   2018-04-11
2442   2018-04-11
2443   2018-04-11
2444   2018-04-11
2445   2018-04-11
2446   2018-04-11
2447   2018-04-11
2448   2018-04-11
2449   2018-04-11
2450   2018-04-11
2451   2018-04-11
2452   2018-04-11
2453   2018-04-11
2454   2018-04-11
2455   2018-04-11
2456   2018-04-11
2457   2018-04-11
2458   2018-04-11
Name: Date, dtype: datetime64[ns]

In [7]:
### 2 ###
    
# Find difference between date last updated and current date, get range of dates
last_date = db['Date'].iloc[-1]
d = pd.Timedelta(today - db['Date'].iloc[-1])
daterange = pd.Series(pd.date_range(db['Date'].iloc[-1], periods = d.days, freq = 'D'))
print(daterange)

0     2018-04-11
1     2018-04-12
2     2018-04-13
3     2018-04-14
4     2018-04-15
5     2018-04-16
6     2018-04-17
7     2018-04-18
8     2018-04-19
9     2018-04-20
10    2018-04-21
11    2018-04-22
12    2018-04-23
13    2018-04-24
14    2018-04-25
15    2018-04-26
16    2018-04-27
17    2018-04-28
18    2018-04-29
19    2018-04-30
20    2018-05-01
21    2018-05-02
22    2018-05-03
23    2018-05-04
24    2018-05-05
25    2018-05-06
26    2018-05-07
27    2018-05-08
28    2018-05-09
29    2018-05-10
         ...    
167   2018-09-25
168   2018-09-26
169   2018-09-27
170   2018-09-28
171   2018-09-29
172   2018-09-30
173   2018-10-01
174   2018-10-02
175   2018-10-03
176   2018-10-04
177   2018-10-05
178   2018-10-06
179   2018-10-07
180   2018-10-08
181   2018-10-09
182   2018-10-10
183   2018-10-11
184   2018-10-12
185   2018-10-13
186   2018-10-14
187   2018-10-15
188   2018-10-16
189   2018-10-17
190   2018-10-18
191   2018-10-19
192   2018-10-20
193   2018-10-21
194   2018-10-

In [8]:
# Define offseason months so don't search those; year accounts for updating between seasons
offseason = ['may', 'june', 'july', 'august', 'september']
months = list(set([datetime.strftime(date, '%B').lower() for date in daterange if datetime.strftime(date, '%B').lower() not in offseason]))
years = [today.year + 1 if month in ['october', 'november', 'december'] else today.year for month in months]
print(list(zip(months, years)))




[('april', 2018), ('october', 2019)]


In [10]:
for month, year in list(zip(months,years)):
    url = f'https://www.basketball-reference.com/leagues/NBA_{year}_games-{month}.html'
    print(url)
    res = requests.get(url, 'html_parser')
    res.raise_for_status()
    soup = bs4.BeautifulSoup(res.text, 'html.parser')
    game_soup = soup.find('table',{'id':'schedule'})
    
    datesoup = game_soup.find_all(attrs = {'data-stat':'date_game'})
    gamedates = [entry.get_text() for entry in datesoup if entry.get_text() != 'Date' ]
    game_dates = list(set([datetime.strptime(date, '%a, %b %d, %Y') for date in gamedates]))
    print(game_dates)

https://www.basketball-reference.com/leagues/NBA_2018_games-april.html
[datetime.datetime(2018, 4, 25, 0, 0), datetime.datetime(2018, 4, 11, 0, 0), datetime.datetime(2018, 4, 1, 0, 0), datetime.datetime(2018, 4, 18, 0, 0), datetime.datetime(2018, 4, 21, 0, 0), datetime.datetime(2018, 4, 9, 0, 0), datetime.datetime(2018, 4, 14, 0, 0), datetime.datetime(2018, 4, 7, 0, 0), datetime.datetime(2018, 4, 16, 0, 0), datetime.datetime(2018, 4, 19, 0, 0), datetime.datetime(2018, 4, 28, 0, 0), datetime.datetime(2018, 4, 29, 0, 0), datetime.datetime(2018, 4, 23, 0, 0), datetime.datetime(2018, 4, 6, 0, 0), datetime.datetime(2018, 4, 17, 0, 0), datetime.datetime(2018, 4, 3, 0, 0), datetime.datetime(2018, 4, 20, 0, 0), datetime.datetime(2018, 4, 4, 0, 0), datetime.datetime(2018, 4, 8, 0, 0), datetime.datetime(2018, 4, 27, 0, 0), datetime.datetime(2018, 4, 22, 0, 0), datetime.datetime(2018, 4, 15, 0, 0), datetime.datetime(2018, 4, 24, 0, 0), datetime.datetime(2018, 4, 26, 0, 0), datetime.datetime(2018,

In [60]:
for date in game_dates:
    if date.month == 4:
        game_dates = [date for date in game_dates if date.day < 14]
    else:
        game_dates = [date for date in game_dates]
# Pull game dates, generate list, turn it into datetime objects
#game_dates = [date for date in game_dates if date in db['Date']]


In [69]:
game_dates = pd.to_datetime([date for date in game_dates if date not in db.Date])

In [70]:
game_dates

DatetimeIndex(['2018-04-03', '2018-04-09', '2018-04-04', '2018-04-07',
               '2018-04-11', '2018-04-10', '2018-04-05', '2018-04-01',
               '2018-04-06', '2018-04-08'],
              dtype='datetime64[ns]', freq=None)

In [80]:
db[db['Date']= pd.

0      2017-10-17
1      2017-10-17
2      2017-10-17
3      2017-10-17
4      2017-10-18
5      2017-10-18
6      2017-10-18
7      2017-10-18
8      2017-10-18
9      2017-10-18
10     2017-10-18
11     2017-10-18
12     2017-10-18
13     2017-10-18
14     2017-10-18
15     2017-10-18
16     2017-10-18
17     2017-10-18
18     2017-10-18
19     2017-10-18
20     2017-10-18
21     2017-10-18
22     2017-10-18
23     2017-10-18
24     2017-10-18
25     2017-10-18
26     2017-10-19
27     2017-10-19
28     2017-10-19
29     2017-10-19
          ...    
2430   2018-04-10
2431   2018-04-10
2432   2018-04-10
2433   2018-04-10
2434   2018-04-10
2435   2018-04-10
2436   2018-04-11
2437   2018-04-11
2438   2018-04-11
2439   2018-04-11
2440   2018-04-11
2441   2018-04-11
2442   2018-04-11
2443   2018-04-11
2444   2018-04-11
2445   2018-04-11
2446   2018-04-11
2447   2018-04-11
2448   2018-04-11
2449   2018-04-11
2450   2018-04-11
2451   2018-04-11
2452   2018-04-11
2453   2018-04-11
2454   201